# Lab 8 – Production LLM Deployment with vLLM (OpenAI-Compatible)

In this lab, you will deploy an optimized LLM using **vLLM**, the industry-standard production inference server. vLLM provides OpenAI-compatible endpoints with advanced features like continuous batching, PagedAttention, and efficient memory management.

## Objectives

- Export an Unsloth-optimized model to HuggingFace format for vLLM deployment
- Deploy the model using vLLM's production-grade OpenAI-compatible server
- Test the endpoint with OpenAI SDK
- Understand production deployment best practices (batching, GPU utilization, scaling)

## Why vLLM for Production?

vLLM is the gold standard for production LLM serving:
- ⚡ **High throughput**: Continuous batching + PagedAttention (24x faster than naive implementations)
- 🎯 **Low latency**: Optimized CUDA kernels
- 📊 **Resource efficient**: vLLM v1 engine (default since v0.8.0) provides 1.5-2x better throughput with auto-optimization
- 🔌 **OpenAI-compatible**: Drop-in replacement for OpenAI API
- 📈 **Production-ready**: Request queuing, multi-GPU support, Prometheus metrics

**⚠️ Important for 2025**: Use vLLM's v1 engine (default) - don't add deprecated v0 flags like `--enable-chunked-prefill` or `--num-scheduler-steps` as they force fallback to slower v0 engine.

This is what you'd actually use in production, not a custom FastAPI wrapper.

In [ ]:
# Install Unsloth using the official auto-install script
# This automatically detects your environment and installs the correct version
!wget -qO- https://raw.githubusercontent.com/unslothai/unsloth/main/unsloth/_auto_install.py | python -

# Alternative manual installation if auto-install fails:
# !pip install --upgrade pip
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

print("✅ Unsloth installation complete! Now restart runtime before proceeding.")
print("⚠️ IMPORTANT: Use GPU runtime, not TPU! Unsloth requires CUDA GPU.")

> **⚠️ CRITICAL IMPORT ORDER**: 
> - Always import `unsloth` FIRST before any other ML libraries
> - This prevents weights/biases initialization errors
> - Example: `from unsloth import FastLanguageModel` then `import torch`


### Step 1: Build an OpenAI-compatible FastAPI server

**Documentation:**
- FastAPI tutorial: https://fastapi.tiangolo.com/tutorial/
- Pydantic models: https://docs.pydantic.dev/latest/
- OpenAI Chat Completions API: https://platform.openai.com/docs/api-reference/chat


In [ ]:
# TODO: Import (CRITICAL: Import unsloth FIRST!) necessary libraries (FastAPI, Pydantic, torch, unsloth)# TODO: Load your optimized model using FastLanguageModel.from_pretrained()# Replace with your own checkpoint if necessary# TODO: Define Pydantic models for the API:#   - ChatMessage (role: str, content: str)#   - ChatRequest (model: str, messages: List[ChatMessage], temperature: float, max_tokens: int, stream: bool)#   - Choice (index: int, message: ChatMessage, finish_reason: str)#   - ChatResponse (id: str, object: str, choices: List[Choice], model: str)# TODO: Create FastAPI app instance# TODO: Create POST endpoint at "/v1/chat/completions" that:#   1. Receives ChatRequest#   2. Extracts the last message content as prompt#   3. Tokenizes the prompt#   4. Generates response using model.generate() with torch.inference_mode()#   5. Decodes the output#   6. Returns ChatResponse with proper structure# TODO: Add instructions for running the server# Hint: uvicorn.run(app, host="0.0.0.0", port=8000)

### Step 2: Test the API with an OpenAI client

**Documentation:**
- OpenAI Python library: https://github.com/openai/openai-python
- Using custom base URLs: https://github.com/openai/openai-python#using-a-custom-base-url


In [ ]:
# This cell shows how to test your FastAPI service once it's running locally.
# Please start the FastAPI server in a separate process or notebook cell, then run the following code:

# TODO: Import OpenAI client

# TODO: Create client with custom base_url pointing to your local server
# Example: client = OpenAI(base_url="http://localhost:8000/v1", api_key="dummy-key")

# TODO: Make a chat completion request with:
#   - model name (can be arbitrary)
#   - messages list with a user message
#   - Example prompt: "Hello! Can you explain the concept of quantization in LLMs?"

# TODO: Print the response content

# Note: The above test simulates how an OpenAI client can interact with your service. Adjust the port and base URL accordingly.


## Production Deployment with Docker + vLLM

To deploy to AWS ECS, containerize using the official vLLM image:

**Documentation:**
- vLLM documentation: https://docs.vllm.ai/
- vLLM CLI guide: https://docs.vllm.ai/en/latest/cli/index.html
- Docker best practices: https://docs.docker.com/develop/dev-best-practices/

### Production-Grade Dockerfile

```dockerfile
# Use official vLLM image (includes CUDA, PyTorch, and all dependencies)
FROM vllm/vllm-openai:latest

# Set working directory
WORKDIR /app

# Copy your fine-tuned model
COPY ./vllm_model /app/model

# Expose vLLM port
EXPOSE 8000

# Health check for container orchestration
HEALTHCHECK --interval=30s --timeout=10s --start-period=60s --retries=3 \
  CMD curl -f http://localhost:8000/health || exit 1

# Start vLLM OpenAI-compatible server (v1 engine - auto-optimized)
# ⚠️ DON'T add --enable-chunked-prefill or --num-scheduler-steps
# These force v0 fallback and reduce performance by 30-50%
CMD ["python", "-m", "vllm.entrypoints.openai.api_server", \
     "--model", "/app/model", \
     "--host", "0.0.0.0", \
     "--port", "8000", \
     "--dtype", "float16", \
     "--max-model-len", "4096", \
     "--gpu-memory-utilization", "0.9"]
```

### vLLM v1 Engine (2025)

**Key improvements since v0.8.0:**
- 1.5-2x better throughput (automatic)
- Auto-optimized chunked prefill
- Auto-tuned scheduler steps
- Better memory efficiency

**What NOT to do:**
```bash
# ❌ BAD - Forces v0 fallback
--enable-chunked-prefill
--num-scheduler-steps 10

# ✅ GOOD - Let v1 auto-optimize
# (just omit these flags)
```

### Build and Deploy

```bash
# Build Docker image
docker build -t my-llm-service:latest .

# Test locally with GPU
docker run --gpus all -p 8000:8000 my-llm-service:latest

# Push to AWS ECR
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin <account>.dkr.ecr.us-east-1.amazonaws.com
docker tag my-llm-service:latest <account>.dkr.ecr.us-east-1.amazonaws.com/my-llm-service:latest
docker push <account>.dkr.ecr.us-east-1.amazonaws.com/my-llm-service:latest
```

### Production Scaling Strategy

1. **Horizontal Scaling**: Multiple ECS tasks behind Application Load Balancer
2. **Vertical Scaling**: Use larger GPU instances (g5.xlarge → g5.2xlarge → g5.12xlarge)
3. **Multi-Model Serving**: vLLM can serve multiple LoRA adapters efficiently
4. **Auto-scaling**: Based on `vllm:num_requests_waiting` metric (not CPU)

## Reflection

### Performance Analysis
- Did the OpenAI-compatible endpoint respond as expected when tested with an OpenAI client?
- Compare the latency of vLLM versus running the model locally in notebook cells. What's the overhead?
- What throughput (tokens/second) does vLLM achieve with continuous batching?

### Production Deployment Learnings

**Why vLLM over custom FastAPI?**
- vLLM provides continuous batching, PagedAttention, and optimized CUDA kernels
- 24x higher throughput than naive PyTorch implementations
- vLLM v1 engine (default since v0.8.0) adds another 1.5-2x improvement with zero config

**vLLM v1 Engine (2025):**
- **Auto-optimized**: No need for manual `--enable-chunked-prefill` or `--num-scheduler-steps`
- **Performance**: 1.5-2x better throughput vs v0 automatically
- **Memory efficient**: Smarter KV cache management
- **Lower latency**: Optimized scheduling algorithms
- **⚠️ Don't use v0 flags**: They force fallback to slower v0 engine

**Production hardening checklist:**
- ✅ Authentication: Add API key validation or OAuth2
- ✅ Rate limiting: Use AWS API Gateway or NGINX
- ✅ Monitoring: vLLM exposes Prometheus metrics at `/metrics`
  - `vllm:num_requests_waiting` - for autoscaling
  - `vllm:time_to_first_token_seconds` - latency
  - `vllm:avg_generation_throughput_toks_per_s` - throughput
- ✅ Auto-scaling: CloudWatch alarms based on queue depth (not CPU)
- ✅ Cost optimization: Use Spot instances for 70% savings

**Scaling considerations:**
- **<100 QPS**: Single vLLM instance (g5.xlarge)
- **100-1k QPS**: Multiple instances behind ALB
- **>1k QPS**: Ray Serve with distributed orchestration
- **Multi-tenancy**: Use LoRA adapters (10-100x more memory efficient)

**When to use what:**
- Development/prototyping: Direct model loading
- Production < 100 QPS: vLLM single instance
- Production > 100 QPS: vLLM with horizontal scaling + load balancer
- Ultra-low latency (<50ms): TensorRT-LLM with optimized kernels
- Enterprise scale: Ray Serve + vLLM + LoRA multi-tenancy